## Build Table Business vs. Categories

In [5]:
import json
import sqlite3

import pandas as pd
import numpy as np
#from sqlalchemy import create_engine
from collections import defaultdict

import sys  

#reload(sys)  
sys.setdefaultencoding('utf8')

In [7]:
#############
# Might need to change path to read the file in your local machine 
business = pd.read_json('yelp_dataset_challenge_round9/yelp_academic_dataset_business.json', \
                        lines=True, encoding='utf8')
business.head(3)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,type
0,"227 E Baseline Rd, Ste J2","[BikeParking: True, BusinessAcceptsBitcoin: Fa...",0DI8Dt2PJp07XkVvIElIcQ,"[Tobacco Shops, Nightlife, Vape Shops, Shopping]",Tempe,"[Monday 11:0-21:0, Tuesday 11:0-21:0, Wednesda...",0,33.378214,-111.936102,Innovative Vapors,,85283,17,4.5,AZ,business
1,495 S Grand Central Pkwy,"[BusinessAcceptsBitcoin: False, BusinessAccept...",LTlCaCGZE14GuaUXUGbamg,"[Caterers, Grocery, Food, Event Planning & Ser...",Las Vegas,"[Monday 0:0-0:0, Tuesday 0:0-0:0, Wednesday 0:...",1,36.192284,-115.159272,Cut and Taste,,89106,9,5.0,NV,business
2,979 Bloor Street W,"[Alcohol: none, Ambience: {'romantic': False, ...",EDqCEAGXVGCH4FJXgqtjqg,"[Restaurants, Pizza, Chicken Wings, Italian]",Toronto,"[Monday 11:0-2:0, Tuesday 11:0-2:0, Wednesday ...",1,43.661054,-79.429089,Pizza Pizza,Dufferin Grove,M6H 1L5,7,2.5,ON,business


In [8]:
df_c = business[['business_id','categories']]
df_c.head(3)

,business_id,categories
0,0DI8Dt2PJp07XkVvIElIcQ,"[Tobacco Shops, Nightlife, Vape Shops, Shopping]"
1,LTlCaCGZE14GuaUXUGbamg,"[Caterers, Grocery, Food, Event Planning & Ser..."
2,EDqCEAGXVGCH4FJXgqtjqg,"[Restaurants, Pizza, Chicken Wings, Italian]"


In [9]:
len(df_c)

144072

In [10]:
# Build table of business and categories 
def yes_no(df,col): # data, which col to expand(name)
    """
    the function takes a dataframe and column name
    df: dataframe
    col: name of the column containing a list of words(i.e.categories)
    returns a dictionary with 
        keys--all the words; 
        values as 0/1--0 if the business's categories do not include this word; 1 if it does
    """
    
    dic = defaultdict(list) # dictionary whose values are lists.

    for i in range(len(df)):
        #lst = df.iloc[:,col][i].encode('ascii','ignore')[3:-2].split('\', u\'') # convert unicode list to list of strings
        lst_u = df[col][i]   
        
        try:     
            lst = []
            for j in range(len(lst_u)):
                lst.append(str(lst_u[j]))

            for k in dic.keys(): 
                if k in lst:
                    dic[k].append(1)
                elif k not in lst:
                    dic[k].append(0)
                else:
                    pass
            for c in lst:
                if c not in dic.keys():
                    zeros = [0] * i
                    dic[c] = zeros + [1]
        
        except:
            # if a business does not have "categoriy" items, add 0 for all keys
            for k in dic.keys():
                dic[k].append(0)
            continue


    return dic

In [11]:
##!! Caution!! takes some time to run
dic_all = yes_no(df_c, 'categories')

In [12]:
len(df_c)

144072

In [13]:
df_cat = pd.DataFrame.from_dict(dic_all)
df_cat.index = df_c.business_id
df_cat.head()

,& Probates,3D Printing,ATV Rentals/Tours,Acai Bowls,Accessories,Accountants,Acne Treatment,Active Life,Acupuncture,Addiction Medicine,...,Wine Tours,Wineries,Wok,Women's Clothing,Workers Compensation Law,Wraps,Yelp Events,Yoga,Ziplining,Zoos
business_id,,,,,,,,,,,,,,,,,,,,,
0DI8Dt2PJp07XkVvIElIcQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTlCaCGZE14GuaUXUGbamg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
EDqCEAGXVGCH4FJXgqtjqg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cnGIivYRLxpF7tBVR_JwWA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cdk-qqJ71q6P7TJTww_DSA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Check
df_cat.loc['SWkmyhQBV1A7Qu4LRHoVXQ',:][df_cat.loc['SWkmyhQBV1A7Qu4LRHoVXQ',:]==1]

Acupuncture         1
Beauty & Spas       1
Chiropractors       1
Health & Medical    1
Massage             1
Massage Therapy     1
Physical Therapy    1
Name: SWkmyhQBV1A7Qu4LRHoVXQ, dtype: int64

In [ ]:
# Save as pickle file
df_cat.to_pickle('Yelp_Business_Catergory_Table.p')

## Select Categories representing resturants

In [2]:
import pandas as pd
import numpy as np

In [4]:
## load all cat from pickle
df = pd.read_pickle('Yelp_Business_Catergory_Table.p')

In [5]:
df.shape

(144072, 1191)

In [8]:
for col in df.columns[:5]:
    print col

& Probates
3D Printing
ATV Rentals/Tours
Acai Bowls
Accessories


In [9]:
# pick columns
columns_food = []
columns_culture = []
columns_not_sure = []
columns_unrelated = []

for col in df.columns:
    print (col)
    food = raw_input('about food?   ')
    if food == 'y':
        columns_food.append(col)
    
    culture = raw_input('about culture? ')
    if culture == 'y':
        columns_culture.append(col)
     
    elif (food != 'y') & (culture != 'y'):
        very_sure = raw_input('nothing to do with restuarant  ?')
        if very_sure == 'y':
            columns_unrelated.append(col)
        else:
            columns_not_sure.append(col)
        

& Probates
about food?   
about culture? 
nothing to do with restuarant  ?y
3D Printing
about food?   
about culture? 
nothing to do with restuarant  ?y
ATV Rentals/Tours
about food?   
about culture? 
nothing to do with restuarant  ?y
Acai Bowls
about food?   y
about culture? 
Accessories
about food?   
about culture? 
nothing to do with restuarant  ?y
Accountants
about food?   
about culture? 
nothing to do with restuarant  ?y
Acne Treatment
about food?   
about culture? 
nothing to do with restuarant  ?y
Active Life
about food?   
about culture? 
nothing to do with restuarant  ?y
Acupuncture
about food?   
about culture? 
nothing to do with restuarant  ?y
Addiction Medicine
about food?   
about culture? 
nothing to do with restuarant  ?y
Adoption Services
about food?   
about culture? 
nothing to do with restuarant  ?y
Adult
about food?   
about culture? 
nothing to do with restuarant  ?y
Adult Education
about food?   
about culture? 
nothing to do with restuarant  ?y
Adult Entertai

In [52]:
columns_food

['Rotisserie Chicken',
 'Tapas/Small Plates',
 'Fondue',
 'Buffets',
 'Gluten-Free',
 'Curry Sausage',
 'Swiss Food',
 'Coffeeshops',
 'Creperies',
 'Sushi Bars',
 'Tea Rooms',
 'Hotel bar',
 'Pretzels',
 'Speakeasies',
 'Dim Sum',
 'Dive Bars',
 'Pasta Shops',
 'Juice Bars & Smoothies',
 'Bistros',
 'Salad',
 'Gelato',
 'Patisserie/Cake Shop',
 'Food Delivery Services',
 'Macarons',
 'Cocktail Bars',
 'Imported Food',
 'Restaurants',
 'Steakhouses',
 'Japanese Curry',
 'Beach Bars',
 'Meat Shops',
 'Desserts',
 'Delis',
 'Bed & Breakfast',
 'Falafel',
 'Noodles',
 'Beverage Store',
 'Internet Cafes',
 'Irish Pub',
 'Chicken Wings',
 'Breakfast & Brunch',
 'Beer',
 'Food Stands',
 'Beer Hall',
 'Themed Cafes',
 'Coffee & Tea',
 'Cheesesteaks',
 'Supper Clubs',
 'Tapas Bars',
 'Lounges',
 'Fish & Chips',
 'Cafes',
 'Izakaya',
 'Food Banks',
 'Bar Crawl',
 'Burgers',
 'Island Pub',
 'Teppanyaki',
 'Beer Bar',
 'Ice Cream & Frozen Yogurt',
 'Bars',
 'Kebab',
 'Food Trucks',
 'Whiskey Bars

In [54]:
columns_culture

['Salvadoran',
 'Tapas/Small Plates',
 'Arabian',
 'Canadian (New)',
 'Belgian',
 'Bulgarian',
 'French',
 'Szechuan',
 'Armenian',
 'Haitian',
 'Dominican',
 'Filipino',
 'Kombucha',
 'Thai',
 'Italian',
 'Hakka',
 'Mediterranean',
 'Moroccan',
 'Czech',
 'Puerto Rican',
 'Halal',
 'Scottish',
 'Senegalese',
 'Austrian',
 'Caribbean',
 'American (New)',
 'Pan Asian',
 'Palatine',
 'International',
 'Indonesian',
 'Hainan',
 'Hawaiian',
 'Iberian',
 'Slovakian',
 'Argentine',
 'Persian/Iranian',
 'Cuban',
 'Cajun/Creole',
 'Minho',
 'Eastern European',
 'Uzbek',
 'Cantonese',
 'Scandinavian',
 'Greek',
 'Burmese',
 'African',
 'Singaporean',
 'Chinese',
 'Asian Fusion',
 'Bavarian',
 'Ethiopian',
 'Alsatian',
 'Turkish',
 'Middle Eastern',
 'Bangladeshi',
 'Indian',
 'Latin American',
 'Brazilian',
 'Korean',
 'Pakistani',
 'Shanghainese',
 'Hungarian',
 'Swabian',
 'Southern',
 'Russian',
 'Afghan',
 'Serbo Croatian',
 'Malaysian',
 'Ukrainian',
 'Modern European',
 'Hong Kong Style C

In [53]:
columns_unrelated

['Roofing',
 'Truck Rental',
 'Drugstores',
 'ATV Rentals/Tours',
 'Well Drilling',
 'Family Practice',
 'Towing',
 'Dog Parks',
 'Process Servers',
 'Boating',
 'Auto Insurance',
 'Child Care & Day Care',
 'Endodontists',
 'Dumpster Rental',
 'Carousels',
 'Toxicologists',
 'Organic Stores',
 'Art Museums',
 'Life Coach',
 'Architects',
 'Paint-Your-Own Pottery',
 'Contract Law',
 'Sandblasting',
 'Climbing',
 'Pet Boarding/Pet Sitting',
 'Grocery',
 'Mortgage Lenders',
 'Pet Adoption',
 'Interval Training Gyms',
 'Country Dance Halls',
 'Cannabis Tours',
 'Baseball Fields',
 'Adult Education',
 'Knife Sharpening',
 'Roof Inspectors',
 'Pathologists',
 'Motorsport Vehicle Repairs',
 'Vascular Medicine',
 'Community Service/Non-Profit',
 'Bowling',
 'Commercial Truck Dealers',
 'Waterproofing',
 'DJs',
 'Formal Wear',
 'Hiking',
 'Indoor Playcentre',
 'Vinyl Siding',
 'General Festivals',
 'Wedding Chapels',
 'Bankruptcy Law',
 'Outdoor Gear',
 'Anesthesiologists',
 'Tanning',
 'Mediat

In [42]:
### post adjustment ###
# columns_food.append('___')
# columns_unrelated.remove('____')

In [60]:
# Avoid repetitive words
columns_food= list(set(columns_unrelated))

In [59]:
# Save to text file
thefile = open('cat_unrelated.txt', 'w')

for item in columns_unrelated:
    thefile.write("%s\n" % item)

In [61]:
### might need to extract Cultural words... ###